# Recommender System

필요한 라이브러리들을 import 합니다.

In [11]:
import networkx as nx
import random
import pandas as pd

# 그래프를 입력받는 과정에서, neo4j와 networkX의 차이로 인해 발생하는 (기능상 문제 없는) 오류 출력을 막습니다.
import warnings
warnings.filterwarnings('ignore')

# Import Graph

초기에 추천 시스템을 구축할 때, neo4j graph local desktop DB를 활용하였습니다.

하지만 티쓰리큐 플랫폼에 업로드를 실시하며 이를 사용할 수 없게 되어, graph DB에 저장되어 있는 graph를 'neo4jdata.graphml' file로 export하였고, 이를 이용하여 추천을 진행하기 위해 export된 파일읍 입력받아 networkx library를 활용하여 그래프를 그려야 합니다.

In [12]:
# Path to your GraphML file
graphml_file_path = 'neo4jdata.graphml'

# Read the GraphML file
# GraphML file를 networkX에서 사용하는 graph G로 읽어들입니다.
G = nx.read_graphml(graphml_file_path)

# Warning은 무시해도 상관 없습니다.
# 기능 상에 문제가 있는 것이 아닌, neo4j에서 supercategoryID를 설정할 때, 이를 string datatype으로 설정하였는데, 
# 이를 netwokx에서 사용할 수 없어서 이를 바꾸겠다는 warning입니다.

# Recommendation 관련 함수들

0. video_subject category conversion <br>
   : video 전체의 주제인 video_subject를 적절한 item category에 배정을 해주는 것입니다. <br>
   : 이는 영상 주제가 고양이 (1), 강아지 (0), 고양이&강아지 (-1)인 경우로 표기합니다. <br>
   : 이는 쿠팡 상품 카테고리를 참고하였을 때, 고양이&강아지인 카테고리가 없기 때문에 저희 조의 기준에 따라 다시 설정했기 때문에 필요한 부분입니다.

In [13]:
def convert_category(detection_category, video_subject):
    if video_subject == 1:
        # Mapping for video_subject = 1 (고양이)
        mapping = {2: 118920, 3: 118923, 4: 119567, 5: 119567, 6: 119571, 7: 119562, 8: 119564, 9: 157054}
    elif video_subject == 0:
        # Mapping for video_subject = 0 (강아지)
        mapping = {2: 118920, 3: 118923, 4: 119567, 5: 119567, 6: 119571, 7: 118920, 8: 118922, 9: 118926}
    else:
        # Mapping for video_subject = -1 (공통)
        mapping = {2: 118920, 3: 118923, 4: 119567, 5: 119567, 6: 119571, 7: 238482, 8: 238486, 9: 275980}
    
    return mapping.get(detection_category)

1. userID 또는 categoryID를 각각 해당하는 node의 ID로 변환해주는 함수 <br>
: neo4j에서는 각 node의 ID를 user node는 userID, category node는 categoryID와 같이 설정을 하였습니다. <br>
다만, 이를 neo4jdata.graphml로 export를 하면, 이를 무시하고 정해진 순서대로 처음부터 다시 node와 relation의 ID를 설정하게 됩니다. <br>
그렇기 때문에 neo4j DB를 활용하여 추천 시스템을 구축한 코드를 활용하며 최대한 유사하도록 변환을 하기 위해서는 node의 ID를 userID 또는 categoryID와 같이 변환을 해줘야 합니다.

In [14]:
# userID 또는 categoryID로 각 node를 찾는 함수입니다.
# 이는 graphDB에서 networkX로 옮겨 오면서 발생하는 ID 차이로 인해 사용합니다.
def find_node_by_attribute(attr, value):
    for node, data in G.nodes(data=True):
        if data.get(attr) == value:
            return node
    return None

2. Category 추천 시스템 <br>
: 사물 인식의 결과를 받아서, tab 2, 3에서 item 추천에 사용될 category를 추천해줍니다. <br>
: 입력으로 grpah G, 사용자의 user_id, 사물 인식에서 나온 key_category_id를 활용합니다.

In [15]:
def category_recommender(user_id, key_category_id):
    # Step 1: Find the keySupercategory of the keyCategory
    key_category_node = find_node_by_attribute('categoryID', key_category_id)
    key_supercategory_node = None
    for u, v, d in G.edges(key_category_node, data=True):
        if d.get('label') == 'CATEGORY_BELONGS_TO':
            key_supercategory_node = v
            break

    if not key_supercategory_node:
        raise ValueError("Key supercategory not found")
    
    # Step 2: Find similar users
    target_user_node = find_node_by_attribute('userID', str(user_id))
    # Similar user가 없는 경우에는 이를 빈 list로 해줘야 한다. 
    # 이렇게 하지 않으면 모든 edges를 다 가져오기 때문에 문제가 생긴다.
    if target_user_node:
        similar_users = [v for u, v in G.edges(target_user_node) if G[u][v].get('label') == 'SIMILAR']
    else:
        similar_users = []
    
    # Initialize recommended_categories
    recommended_categories = {}

    # If there are similar users, proceed with collaborative filtering
    if similar_users:
        for su in similar_users:
            for u, v, d in G.edges(su, data=True):
                if d.get('label') == 'PREFERENCE':
                    if any(d.get('label') == 'CATEGORY_BELONGS_TO' for _, _, d in G.edges(v, data=True)):
                        category_id = G.nodes[v].get('categoryID')
                        recommended_categories[category_id] = recommended_categories.get(category_id, 0) + 1

        # Sort and get top 2 categories
        sorted_categories = sorted(recommended_categories, key=recommended_categories.get, reverse=True)[:2]

    else:
        # If no similar users, pick 2 random categories from the same supercategory
        potential_categories = [u for u, v, d in G.edges(key_category_node, data=True) 
                                if d.get('label') == 'CATEGORY_BELONGS_TO' and G.nodes[u].get('categoryID') != key_category_id]
        # no duplicates
        potential_categories = list(set(potential_categories))

        # Randomly select up to 2 categories
        selected_nodes = random.sample(potential_categories, 2) 
        sorted_categories = [G.nodes[v].get('categoryID') for v in selected_nodes]

    return [int(x) for x in sorted_categories[:2]]


3. Item 추천 시스템 <br>
: tab 2, 3에서 주어진 category와 사용자 정보를 바탕으로 item을 추천해줍니다. <br>
: 입력으로 graph G, 사용자의 user_id, category 추천에서 나온 category를 사용합니다.

In [16]:
def item_recommender(user_id, category_id):
    # Step 1: Find similar users and their rated items
    target_user_node = find_node_by_attribute('userID', str(user_id))
    if target_user_node:
        similar_users = [v for u, v in G.edges(target_user_node) if G[u][v].get('label') == 'SIMILAR']
    else:
        similar_users = []
    item_scores = {}

    print(similar_users)
    # items that target_user already bought
    target_user_items = [v for u, v, d in G.edges(target_user_node, data=True) if d.get('label') == 'RATED']
    for su in similar_users:
        print(type(su))
        for u, item, data in G.edges(data=True):
            if data.get('label') == 'RATED' and item not in target_user_items and u == su:
                item_data = G.nodes[item]
                # 하나의 item이 동시에 여러 category에 속할 수 있으므로 (['공통']의 경우), 그것을 반영합니다
                category_ids = [G.nodes[v].get('categoryID') for u, v, d in G.edges(item, data=True) if d.get('label') == 'BELONGS_TO']
                if str(category_id) in category_ids:
                    # item 평점을 가져옵니다. 이로 우선순위를 부여합니다.
                    rating = data.get('Rating', 0)
                    item_scores[item] = item_scores.get(item, 0) + float(rating)
        print(item_scores)
        print('category_ids:')
        print(category_ids)
    # Step 2: item을 rating을 기준으로 높은 것을 추천해줍니다.
    top_items = sorted(item_scores, key=item_scores.get, reverse=True)[:4]
    print(top_items)
    # Step 3: Cold Start Problem
    # similar user가 없는 경우, 또는 추천하는 item이 부족한 경우, 자체적인 score를 계산하여 추천을 합니다.
    # 동일 category 내에서 추천을 해줍니다.
    # Calculate the number of recommendations already found
    num_of_recommendations = len(top_items)
    num_of_needed_recommendations = 4 - num_of_recommendations
    print(num_of_needed_recommendations)

    # Cold Start Problem Handling
    if num_of_needed_recommendations > 0:
         # Find the node that represents the specified category
        category_node = find_node_by_attribute('categoryID', str(category_id))
    
        # Find all item nodes connected to this category node and not in top_items
        all_items_in_category = []
        if category_node:
            for u, v, d in G.edges(data=True):
                #print("Edge from", u, "to", v, "with data:", d)
                if d.get('label') == 'BELONGS_TO' and v == category_node and u not in top_items:
                    all_items_in_category.append(u)
    
        # Calculate a score for each additional item based on NumOfReviews and Rating
        cold_start_scores = {item: float(G.nodes[item].get('NumOfReviews', 0)) * float(G.nodes[item].get('Rating', 0)) for item in all_items_in_category}
        # Select additional items based on the score
        additional_items = sorted(cold_start_scores, key=cold_start_scores.get, reverse=True)[:num_of_needed_recommendations]
        top_items.extend(additional_items)
    
    # Convert node IDs to itemIDs
    recommended_itemIDs = [(G.nodes[item].get('itemID')) for item in top_items]

    return recommended_itemIDs

4. Random 추천 <br>
: tab 4의 경우, 추천에서 신선함을 부여하기 위해 random 추천을 활용하였습니다. <br>
: random recommendation의 경우, 우선 category를 random하게 추천해준 후, 이 category 내에 있는 item을 random하게 추천해줍니다. <br>
: random 추천이지만, tab 2, 3의 category를 다시 추천해주는 것을 방지하도록 하였습니다. <br>
또한, 영상의 주제와 너무 동떨어진 추천을 하는 것을 방지하기 위하여 영상 전체 주제인 'video_subject' 내에서 추천을 진행하도록 하였습니다.

In [17]:
def random_recommender(tab2_cat, tab3_cat, video_subject):
    # Find all supercategories belonging to the given metacategory
    # Find the metacategory node
    metacategory_node = None
    for node, data in G.nodes(data=True):
        if data.get('metacategoryID') == str(video_subject):
            metacategory_node = node
    
    # Find all supercategories under this metacategory
    supercategories = [u for u, v, d in G.edges(data=True) if d.get('label') == 'SUPERCATEGORY_BELONGS_TO' and v == metacategory_node]

    # Find all categories within these supercategories, excluding tab2_cat and tab3_cat
    potential_categories = []
    for sc in supercategories:
        categories = [u for u, v, d in G.edges(data=True) if d.get('label') == 'CATEGORY_BELONGS_TO' and v == sc]
        potential_categories.extend(categories)

    # Filter categories and randomly select one
    potential_categories = [cat for cat in potential_categories if G.nodes[cat].get('categoryID') not in [tab2_cat, tab3_cat]]
    tab4_cat = random.choice(potential_categories) if potential_categories else None
    
    # Randomly recommend items from the selected category
    tab4_items = []
    if tab4_cat:
        items_in_category = [u for u, v, d in G.edges(data=True) if d.get('label') == 'BELONGS_TO' and v == str(tab4_cat)]
        tab4_items = random.sample(items_in_category, min(len(items_in_category), 4))

    # Convert node IDs to itemIDs
    tab4_item_ids = [int(G.nodes[item].get('itemID')) for item in tab4_items]

    return int(G.nodes[tab4_cat].get('categoryID')), tab4_item_ids

# Workflow

영상 분석을 통한 사물 인식의 결과를 바탕으로, 추천을 진행하는 것 까지의 과정은 다음과 같습니다.

1. 영상에서 detection_category와 video_subject를 input으로 받고, 추가적으로 userID를 같이 input으로 받습니다. <br>
   최종 input은 다음과 같습니다: <br>
   input = {'detection_category':detection_category, 'userID':userID, 'video_subject':video_subject}

2. 이 input을 이용하여 category_recommender에서 category 추천을 해줍니다. <br>
   category_recommender의 output은 다음과 같습니다: <br>
   tab2_cat, tab3_cat = category_recommender(userID, key_category)

3. category_rec_output을 이용해서 item 추천을 진행해줍니다. <br>
   tab2_cat, tab3_cat을 각각 이용해서 tab2_item, tab3_item을 output으로 얻습니다. <br>
   item_recommender의 output은 다음과 같습니다: <br>
   tab2_item = item_recommender(userID, tab2_cat) <br>
   tab3_item = item_recommender(userID, tab3_cat)

4. tab4에 대한 random recommendation을 통해 item을 추천해줍니다. <br>
   tab2_cat, tab3_cat, 사물 인식의 결과로 나온 video_subject를 이용하여 tab4_cat, tab4_item을 output으로 얻습니다. <br>
   random_recommender의 output은 다음과 같습니다: <br>
   tab4_cat, tab4_item = random_recommender(tab2_cat, tab3_cat, video_subject)

In [18]:
# 예제
detection_category = 7
userID = 10
video_subject = 0

input = {'detection_category':detection_category, 'userID':userID, 'video_subject':video_subject}

In [19]:
def main(input_data):

    #추천을 위한 변수들을 준비합니다.
    detection_category = int(input_data["detection_category"])
    userID = int(input_data["userID"])
    video_subject = int(input_data["video_subject"])
    key_category = convert_category(detection_category, video_subject)
    
    #tab2, 3 카테고리 추천
    tab2_cat, tab3_cat = category_recommender(userID, key_category)

    #tab2, 3 item 추천
    tab2_item = item_recommender(userID, tab2_cat)
    tab3_item = item_recommender(userID, tab3_cat)

    #tab4 카테고리, item 추천
    tab4_cat, tab4_item = random_recommender(tab2_cat, tab3_cat, video_subject)

    #최종 결과: list 형태
    # [[tab2_cat, tab3_cat, tab4_cat], tab2_item, tab3_item, tab4_item]
    category_list = [tab2_cat, tab3_cat, tab4_cat]
    item_list = [tab2_item, tab3_item, tab4_item]
    result = [category_list] + item_list

    print(result)
    return result

In [20]:
main(input)

['n842', 'n527', 'n562', 'n546']
<class 'str'>
{'n49664': 5.0}
category_ids:
['509024']
<class 'str'>
{'n49664': 5.0, 'n53090': 1.0}
category_ids:
['381540']
<class 'str'>
{'n49664': 5.0, 'n53090': 1.0}
category_ids:
['486342']
<class 'str'>
{'n49664': 5.0, 'n53090': 1.0, 'n48687': 4.0, 'n51192': 5.0}
category_ids:
['486342']
['n49664', 'n51192', 'n48687', 'n53090']
0
['n842', 'n527', 'n562', 'n546']
<class 'str'>
{'n49666': 5.0}
category_ids:
['509024']
<class 'str'>
{'n49666': 5.0}
category_ids:
['381540']
<class 'str'>
{'n49666': 5.0}
category_ids:
['486342']
<class 'str'>
{'n49666': 5.0, 'n54012': 5.0}
category_ids:
['486342']
['n49666', 'n54012']
2
[[238486, 509024, 445731], ['39290', '39409', '20251', '20831'], ['52126', '52377', '52092', '52331'], [3304, 3333, 3668, 3718]]


[[238486, 509024, 445731],
 ['39290', '39409', '20251', '20831'],
 ['52126', '52377', '52092', '52331'],
 [3304, 3333, 3668, 3718]]